# Exercise 4: Thermostatic HVAC Control

## Overview

In this exercise, we design a simple thermostatic controller for heating a single-zone building modeled with an interpretable $RC$ model (see Exercise 3).

## Set up

Load the required libraries and 10-minute measurement data from an occupied, semi-detached house in the UK [1] (change the ```data_path``` variable accordingly).

[1] Hollick, F; Wingfield, J; (2018) Two periods of in-situ measurements from an occupied, semi-detached house in the UK [Dataset]. 10.14324/000.ds.10087216.

In [1]:
# Import libraries
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# Plotting figures default
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (10,4)

In [2]:
# Load measurement data
data_path = 'C:\\Users\\ucbva19\\Git projects\\BENV0092\\data raw'
raw_data = pd.read_csv(f'{data_path}\\10mins_solpap_2016.csv', index_col = 0)
raw_data.index = pd.to_datetime(raw_data.index, format="%d/%m/%Y %H:%M")

# Fill missing data with linear interpolation
data = raw_data.copy().interpolate('linear')
assert(data.isna().all().sum() == 0)

# Lead T_in values (target variable)
data['T_Average_lead_1'] = data['T_Average (degC)'].shift(-1)
data['T_diff'] = data['T_External (degC)'] - data['T_Average (degC)']

# Drop NaNs created from the shift operator
data = data.dropna()

# discretization step at 10 minutes
dt_h = 1/6

T_in  = data['T_Average (degC)'].values          # °C
T_out = data['T_External (degC)'].values         # °C
u     = data['P_tot (W)'].values        # kW
t_idx = data.index

## RC Model

As in Exercise 3, we use a single-zone linear model, *ignoring solar radiation* gains for simplicity:
<center>
$
    \Delta T_k := 
T_{k+1}-T_{k}
= \alpha_1(T_{\mathrm{ext},k} - T_{k})
+ \alpha_2 u_k + w_k,
$    
</center>

Where:
- $T_k$ : indoor temperature (°C), **state**
- $u_k$ : heating power input (kW), **control**
- $T_{\mathrm{ext}, k}$ : outdoor temperature forecast (°C), **disturbance**
- $w_k$ : process noise (°C), modeling unmodeled gains/losses

As $\alpha_1$ increases, the envelope becomes ``leakier,'' whereas as $\alpha_2$, heat gains increase.
For this exercise, we consider the following values
- $\alpha_1 = 0.00088969,$
- $\alpha_{2} = 0.00674676,$
  
which correspond to a typical small/medium dwelling.

---

## Control Policy

The controller works takes as input a reference setpoint, $T_{\textrm{ref}}$, for the thermostat, which turns the heating on or off (power is zero or maximum) given the curent temperature measurement $T_k$, where $k$ is the time step.

**Policy**: For $k=1, \dots, N$:
- If $T_k < T_\textrm{{ref}}$, then set $u_k=u_{\textrm{max}}$.
- Else, $u_k=0$.

In [3]:
###### Closed-loop Thermostat Baseline

# controller parameters
N_steps = 144
T_min = 20
T_max = 23
T_ref = (T_min+T_max)/2
u_max = 8
u_min = 0

# Starting temperature
T0_real = 20

# RC parameters
alpha_1 = 0.00088969
alpha_2 = 0.00674676

# alternative set of parameters
#alpha_1 = 0.0208
#alpha_2 = 0.16

# thermostat setpoint/ reference
T_ref = (T_min + T_max)/2

################ Design a function that implements the control policy
def thermostatic_ref_control():

    u_next = None
    return u_next


############### Insert code here
# Apply the control policy for N_steps
# Visualize the temperature
# Estimate total heating energy


## Step response

We now allow the reference setpoint to change over time.

In [4]:
# thermostat setpoint/ reference
T_ref_k = np.array(40*[21]+60*[22]+44*[20])

# Insert code here, repeat the same tasks when the reference point changes over time
